In [ ]:
using Random

In [ ]:
using BenchmarkTools

In [ ]:
####### 1d Ising
# Metropolis

In [ ]:
mutable type Measurements
    configurations::Confs # first two dim = conf, third dim = n_conf
    energies::Array{Float64, 1}
    magnetization::Array{Float64, 1}
    
    Measurements() = new()
end

In [ ]:
# Wrap everything with a module to allow redefition of type
module MC

"""
Composite type to represent a spin state
"""
mutable struct SpinState
    num_spins::Int
    s::Array{Int8,1}
    energy::Int
    tot_mag::Int
end

"""
Energy
"""
function energy(s)
    n = length(s)
    - sum((s[i] * s[ifelse(i == n, 1, i+1)] for i in 1:n))
end

"""
Total magnetization
"""
total_magnetization(s) = sum(s)

"""
Constructor
"""
function SpinState(s)
    ss = SpinState(length(s), copy(s), energy(s), total_magnetization(s))
    sanity_check(ss)
    ss
end

"""
Sanity check
"""
function sanity_check(ss)
    @assert energy(ss.s) == ss.energy
    @assert total_magnetization(ss.s) == ss.tot_mag
end

"""
Take an object of SpinState as an input and update it in place.
"""
function update!(ss, β, niters)
    min_h = -2
    max_h = 2
    s = ss.s
    n = ss.num_spins
    prob = [1/(1+exp(-2*β*h)) for h in min_h:max_h]
    for iter in 1:niters, i in 1:n
        sl = s[ifelse(i == 1, n, i-1)]
        sr = s[ifelse(i == n, 1, i+1)]
        # h = -2, 0, 2
        h = sl + sr
        si_old = s[i]
        s[i] = ifelse(rand() < prob[h-min_h+1], +1, -1)
        
        # Update observables with O(1) operations
        ss.energy += (si_old - s[i]) * h
        ss.tot_mag += (s[i] - si_old)
    end
end

end
;

In [ ]:
ss = MC.SpinState(s0)
MC.update!(ss, β, niters)
MC.sanity_check(ss)

In [ ]:
num_spins = 100
s0 = rand(Int8[-1, 1], num_spins)
β = 10.0
niters = 10^3

s = copy(s0)

In [ ]:
nsweeps = 10^6
num_spins = 100

s0 = rand(Int8[-1, 1], num_spins)

Ts_mc = range(0.4, 2, length=10)

C_mc = Float64[]
for T in Ts_mc
    ss = MC.SpinState(s0)
    acc = Meas.Accumulator()
    solve!(ss, acc, 1/T, nsweeps, ntherm, interval_meas)
    E = Meas.mean(acc, "E")
    E2 = Meas.mean(acc, "E2")
    push!(C_mc, (E2-E^2)/T^2)
end

In [ ]:
module Meas

struct Accumulator
    count::Dict{String,UInt64}
    data::Dict{String,Any}
end

"""
Constructor
"""
function Accumulator()
    Accumulator(Dict{String,UInt64}(), Dict{String,Any}())
end

"""
Add a sample
"""
function add!(acc::Accumulator, name::String, data)
    if haskey(acc.count, name)
        acc.count[name] += 1
        acc.data[name] += data
    else
        acc.count[name] = 1
        acc.data[name] = copy(data)
    end
end

"""
Compuate mean
"""
function mean(acc::Accumulator, name::String)
    return acc.data[name]/acc.count[name]
end

end


# Perform some tests
let
    acc = Meas.Accumulator()
    Meas.add!(acc, "obs1", ones(4))
    Meas.add!(acc, "obs1", fill(2.0, 4))
    Meas.add!(acc, "obs2", ones(4))
    Meas.add!(acc, "obs2", ones(4))
    @assert Meas.mean(acc, "obs1") ≈ fill(1.5, 4)
    @assert Meas.mean(acc, "obs2") ≈ ones(4)
end

In [ ]:
function solve!(ss, acc, β, nsweeps, ntherm, interval_meas)
    if mod(nsweeps, interval_meas) != 0
        error("nsweeps cannot be divided by interval_meas!")
    end
    
    # Thermalization steps
    MC.update!(ss, β, ntherm)
    
    # Measurement steps
    for imeas in 1:(nsweeps÷interval_meas)
        MC.update!(ss, β, interval_meas)
        Meas.add!(acc, "E", Float64(ss.energy))
        Meas.add!(acc, "E2", Float64(ss.energy)^2)
    end
    MC.sanity_check(ss)
end

nsweeps = 100
interval_meas = 10
ntherm = nsweeps÷10
β = 1.0

In [ ]:
using Plots

In [ ]:
plot(C_mc)
exact_C(T) = 1/(T * cosh(1/T))^2
plot!([exact_C(x) for x in 1:10])

In [ ]:
s0 = rand(Int8[-1, 1], num_spins)

solve!(s0, 0, 1/2, 10^3, nsweeps÷10, 10)

In [ ]:
heatmap()

In [ ]:
using PyPlot
Ts = range(0.1, 2, length=100)
exact_C(T) = 1/(T * cosh(1/T))^2
plt.plot(Ts, exact_C.(Ts), label="Exact")
plt.plot(Ts_mc, C_mc ./ num_spins, label="MC", marker="o")
plt.xlabel(raw"$T$")
plt.ylabel(raw"$C/N$")
plt.xlim([0, nothing])
plt.ylim([0, nothing])
plt.legend()

In [ ]:
# Wolff

In [3]:
######### 2d Ising
using PlotlyJS
using Random

WebIO._IJuliaInit()

In [4]:
abstract type IsingModel end
abstract type Ising_1d <: IsingModel end
abstract type Ising_2d <: IsingModel end
abstract type Ising_3d <: IsingModel end

mutable struct Periodic_2d <: Ising_2d
  num_spins::Int
  state::Array{Int8, 2}
  beta::T where T <: AbstractFloat
  h::S where S <: AbstractFloat
    
  energy::Float64
  magnetization::Float64

  """
  Constructor for `Periodic_2d`.

  # Arguments: 
  - `ns::Int`: Number of spins

  - `beta::T`: Inverse temperature parameter

  - `h::T=0`: External field parameter
  """
  function Periodic_2d(
    ns::Int,
    beta::T,
    h::T=0
  ) where T <: Real
    @assert ns > 0
    s = rand(Int8[-1,1], (ns,ns))
    new(ns, s, beta, h, hamiltonian(s, beta, h), sum(s))
  end
end

In [5]:
function hamiltonian(im::Periodic_2d)::Float64
  pair_sum = 0.0
  site_sum = im.h * sum(im.state)

  for i in 1:im.num_spins
    for j in 1:im.num_spins
      pair_sum += im.state[i,j] * (i == im.num_spins ? im.state[1,j] : im.state[i+1, j])
      pair_sum += im.state[i,j] * (j == im.num_spins ? im.state[i,1] : im.state[i, j+1])
    end
  end

  return (-im.beta * pair_sum) - site_sum
end

function hamiltonian(s, beta, h)::Float64
  pair_sum = 0.0
  site_sum = h * sum(s)
    
  x,y = size(s)

  for i in 1:x
    for j in 1:y
      pair_sum += s[i,j] * (i == x ? s[1,j] : s[i+1, j])
      pair_sum += s[i,j] * (j == x ? s[i,1] : s[i, j+1])
    end
  end

  return (-beta * pair_sum) - site_sum
end

hamiltonian (generic function with 2 methods)

In [6]:
crit_temp = 2.26918531421;

function neighbors(im::Periodic_2d, i,j)
  ns = im.num_spins
  return [CartesianIndex((ifelse(i==1, ns, i-1), j)),
    CartesianIndex((ifelse(i==ns, 1, i+1), j)),
    CartesianIndex((i, ifelse(j==1, ns, j-1))),
    CartesianIndex((i, ifelse(j==ns, 1, j+1)))]
end

function bond_energy(im, i, j)
    nbrs = neighbors(im, i,j)
    
    return -im.state[i,j]*(im.state[nbrs[1]] + im.state[nbrs[2]] + im.state[nbrs[3]] + im.state[nbrs[4]])
end

function Metropolis!(im::T, niters::Int=10^6) where {T <: Ising_2d}
    @assert niters > 0

    ns = im.num_spins

    for _ in 1:niters
        for dummy in 1:ns^2
            i = rand(1:ns)
            j = rand(1:ns)
        
            energy = bond_energy(im, i,j) / im.beta
            de = -2 * (energy + 0)

            if (de <= 0 || rand() < exp(-de / crit_temp))
                im.state[i,j] *= -1
                im.energy += de / (ns^2)
                im.magnetization += 2.0 * im.state[i,j] * im.state[ns, ns]
            end
        end
    end
end

Metropolis! (generic function with 2 methods)

In [7]:
im1 = Periodic_2d(256, 1.0, 0.0)
Metropolis!(im1, 10^2)

In [8]:
im2 = Periodic_2d(256, 0.000398, 0.0)
Metropolis!(im2, 10^2)

In [9]:
im3 = Periodic_2d(256, 100.0, 0.0)
Metropolis!(im3, 10^2)

In [10]:
plot(heatmap(z=im1.state))

data: [
  "heatmap with fields transpose, type, and z"
]

layout: "layout with fields margin and template"

In [11]:
plot(heatmap(z=im2.state))

data: [
  "heatmap with fields transpose, type, and z"
]

layout: "layout with fields margin and template"

In [13]:
plot(heatmap(z=im3.state))

data: [
  "heatmap with fields transpose, type, and z"
]

layout: "layout with fields margin and template"

In [ ]:
lattice = rand(Int8[-1,1], (256, 256))

In [ ]:
function Wolff!(im::Periodic_2d, niters::Int)
    ns = im.num_spins
    i = rand(1:ns)
    j = rand(1:ns)
    
    
end

In [ ]:
a = rand(Float64, (5,1))

In [ ]:
heatmap(a)

In [22]:
mutable struct Periodic_3d <: Ising_3d
  num_spins::Int
  state::Array{Int8, 3}
  beta::T where T <: AbstractFloat
  h::S where S <: AbstractFloat

  """
  Constructor for `IsingPeriodic_1d`.

  # Arguments: 
  - `ns::Int`: Number of spins

  - `beta::T`: Inverse temperature parameter

  - `h::T=0`: External magnetic field parameter
  """
  function Periodic_3d(
    ns::Int,
    beta::T,
    h::T=0
  ) where T <: Real
    @assert ns > 0
    new(ns, rand(Int8[-1, 1], (ns,ns,ns)), beta, h)
  end
end

LoadError: invalid redefinition of constant Periodic_3d

In [ ]:
function neighbors(im::Periodic_1d, i::Int)
  ns = im.num_spins
  return [CartesianIndex(ifelse(i==1, ns, i-1)), CartesianIndex(ifelse(i==ns, 1, i+1))]
end

function bond_energy(im::T, i) where {T <: Ising_1d}
    nbrs = neighbors(im, i)
    return -im.state[i] * (im.state[nbrs[1]] + im.state[nbrs[2]]) / im.beta
end

"""
            
"""
function Metropolis!(im::T, niters::Int=10^6) where {T <: Ising_1d}
  @assert niters > 0 

  ns = im.num_spins

  for _ in 1:niters
    for _ in 1:ns
      i = rand(1:ns)

      energy = bond_energy(im, i)
      de = -2 * (energy + 0)   
                        
    if (de <= 0 || rand() < exp(-de / crit_temp))
        im.state[i] *= -1
    end
    
  end
                        end
end

In [ ]:
imm3 = Periodic_1d(20, 100.0, 0.0)
Metropolis!(imm3, 10^2)

In [ ]:
imm3

In [ ]:
sum(imm3.state)

In [ ]:
imm3.state

In [ ]:
1+1

In [ ]:
using PlotlyJS

In [ ]:
log(1+√2)/2

In [2]:
2 / log(1 + sqrt(2))

2.269185314213022